In [1]:
import os

import pandas as pd
import matplotlib.pyplot as plt
import steplib as msl
%matplotlib inline

In [2]:
PATH = "DataOx\\Hip_100Hz"

all_files = os.listdir(PATH)

print(all_files) 


['P01_hip100.csv', 'P02_hip100.csv', 'P03_hip100.csv', 'P04_hip100.csv', 'P05_hip100.csv', 'P06_hip100.csv', 'P07_hip100.csv', 'P08_hip100.csv', 'P09_hip100.csv', 'P10_hip100.csv', 'P11_hip100.csv', 'P12_hip100.csv', 'P13_hip100.csv', 'P14_hip100.csv', 'P15_hip100.csv', 'P16_hip100.csv', 'P17_hip100.csv', 'P18_hip100.csv', 'P19_hip100.csv', 'P20_hip100.csv', 'P21_hip100.csv', 'P22_hip100.csv', 'P23_hip100.csv', 'P24_hip100.csv', 'P25_hip100.csv', 'P26_hip100.csv', 'P27_hip100.csv', 'P28_hip100.csv', 'P29_hip100.csv', 'P30_hip100.csv', 'P31_hip100.csv', 'P32_hip100.csv', 'P33_hip100.csv', 'P34_hip100.csv', 'P35_hip100.csv', 'P36_hip100.csv', 'P37_hip100.csv', 'P38_hip100.csv', 'P39_hip100.csv']


In [8]:
import numpy as np
end = 60000

def getSteps(file, window_size=20, threshold=1.15):
    file_data = pd.read_csv("DataOx\\Hip_100Hz\\" + file)[0:end]
    expected = np.where(file_data["annotation"] == 1)[0]
    data1 = file_data["x"] ** 2 + file_data["y"] ** 2 + file_data["z"] ** 2
    accelerometer = data1.apply(lambda x: x ** 0.5)
    data2 = msl.apply_filter(accelerometer, [0.0167855014066363,	0.0470217363830679,	0.121859539593060,	0.198782647391950,	0.231101150450572,	0.198782647391950,	0.121859539593060,	0.0470217363830679,	0.0167855014066363])
    data1 = msl.modified_pan_tompkins_scoring(data2, window_size)
    data1 = msl.detect_outliers(data1, threshold)
    data1 = msl.post_processing(data1, 250, 100)
    calculated = np.where(data1 == 1)[0]


    return calculated, expected , data2

In [4]:
def chart(data_in):
    (calculated, expected, accelerometer) = data_in
    x=range(len(accelerometer))
    y=accelerometer
    plt.figure(figsize=(10,5))
    plt.plot(x,y, color='red', marker='o')
    plt.xlabel('index')
    plt.ylabel('magnitude')
    #show x label every 800
    plt.xticks(x[::500])
    plt.xticks(rotation=90)
    #add rectangles
    for e in expected:
        plt.axvline(x =e, color ='g', label ='axvline - full height', ymin=0.5, alpha=0.8)
    for c in calculated:
        plt.axvline(x =c, color ='b', label ='axvline - full height', ymax=0.5, alpha=0.8)
    plt.show()

In [5]:
differences = []
i = 0
for file in all_files:
    (calculated1, expected1, _) = getSteps(file)
    differences.append(len(expected1) - len(calculated1))
    print(i, len(expected1), len(calculated1), file)
    i += 1
print(differences) 

0 675 615 P01_hip100.csv
1 33 10 P02_hip100.csv
2 838 800 P03_hip100.csv
3 25 16 P04_hip100.csv
4 386 377 P05_hip100.csv
5 242 223 P06_hip100.csv
6 852 727 P07_hip100.csv
7 131 88 P08_hip100.csv
8 201 188 P09_hip100.csv
9 0 14 P10_hip100.csv
10 233 111 P11_hip100.csv
11 796 804 P12_hip100.csv
12 890 798 P13_hip100.csv
13 642 646 P14_hip100.csv
14 85 119 P15_hip100.csv
15 494 445 P16_hip100.csv
16 32 45 P17_hip100.csv
17 131 125 P18_hip100.csv
18 71 65 P19_hip100.csv
19 194 175 P20_hip100.csv
20 219 207 P21_hip100.csv
21 763 636 P22_hip100.csv
22 28 19 P23_hip100.csv
23 633 550 P24_hip100.csv
24 103 94 P25_hip100.csv
25 538 486 P26_hip100.csv
26 47 55 P27_hip100.csv
27 26 22 P28_hip100.csv
28 188 171 P29_hip100.csv
29 895 531 P30_hip100.csv
30 433 406 P31_hip100.csv
31 537 472 P32_hip100.csv
32 932 777 P33_hip100.csv
33 0 35 P34_hip100.csv
34 145 64 P35_hip100.csv
35 132 74 P36_hip100.csv
36 69 34 P37_hip100.csv
37 392 297 P38_hip100.csv
38 192 157 P39_hip100.csv
[60, 23, 38, 9, 9, 19, 

In [9]:
%matplotlib qt
chart(getSteps(all_files[28]))

In [7]:
#TODO: new datasets
#TODO: new filter
#TODO   